# 4.3 インストラクションチューニングによる質問応答のファインチューニング（評価コード）

本Notebookでは、「3-1_InstructionTuning.ipynb」でファインチューニングされた文章生成AIモデルを評価します。

## 評価

### (1) ファインチューニング済みモデルの読み込み

学習コードでファインチューニングしたモデルを読み込みます。  
モデルのパスは学習コードに合わせて設定してください。



In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


# トークナイザ・モデルを読み込みます
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-13b-v1.0", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "llm-jp/llm-jp-13b-v1.0",
    load_in_8bit=True, # 計算を効率化させるため、重みをint8として読み込みます
    device_map="auto"
  )

peft_model = PeftModel.from_pretrained(
    model,
    "./training_logs"
)
peft_model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50688, 5120)
        (wpe): Embedding(2048, 5120)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-39): 40 x GPT2Block(
            (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
             

### (2) データセットの前処理

データセットを前処理します。

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset

# JCommonsenseQAをダウンロードし、データフレームに変換する
dataset_qa = load_dataset("shunk031/JGLUE", name="JCommonsenseQA")
train_df_qa = dataset_qa["train"].to_pandas()
test_df_qa = dataset_qa["validation"].to_pandas()

# JCommonsenseQAデータセットの整備
def preprocess_qa_df(df):
  qa_instruction_text = "質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。"
  df["instruction"] = qa_instruction_text
  df["input"] = df.apply(lambda x: f"質問:\n{x['question']}\n選択肢:\n0.{x['choice0']} 1.{x['choice1']} 2.{x['choice2']} 3.{x['choice3']} 4.{x['choice4']}", axis=1)
  df["output"] = df["label"].astype(str)
  df = df[["instruction", "input", "output"]]
  return df

train_df_qa = preprocess_qa_df(train_df_qa)
test_df_qa = preprocess_qa_df(test_df_qa)

# datasetクラスに変換する
dataset_train = Dataset.from_pandas(train_df_qa)
dataset_train = dataset_train.train_test_split(train_size=0.9, seed=42)
dataset_test = Dataset.from_pandas(test_df_qa)

# 推論用にデータセット前処理
def add_text(example):
    example["text"] = f"### 指示:\n{example['instruction']}\n\n### 入力:\n{example['input']}\n\n### 回答:\n"
    return example

dataset_test = dataset_test.map(add_text)

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

### (3) 推論の実行

推論を実行する関数を作成します。

In [ ]:
def generate(prompt, return_full_text=False, max_token=100):
    input_ids = tokenizer.encode(
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device=peft_model.device)
    output_ids = peft_model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.pad_token_id,
        max_length=max_token,
        temperature=0,
        num_return_sequences=1
    )
    if return_full_text:
        return tokenizer.decode(output_ids.tolist()[0])
    output = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1):])
    return output

prompt = """
### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
"""

print(generate(prompt, True))


### 指示:
以下の質問に回答してください。

### 入力:
日本一高い山は？

### 回答:
富士山<EOD|LLM-jp>


### (4) 精度の評価

精度評価を実施します。

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def predict(prompt):
    answer = generate(prompt, False)
    try:
      return int(answer[0])
    except ValueError:
      return -1

y_preds = []
for i in tqdm(range(len(dataset_test))):
    y_preds.append(predict(dataset_test["text"][i]))

y_true = dataset_test["output"]
y_true = [int(i) for i in y_true]

y_preds = np.array(y_preds)
y_true = np.array(y_true)

print(f"accuracy : {accuracy_score(y_true, y_preds):.3f}")

100%|██████████| 1119/1119 [04:37<00:00,  4.03it/s]

accuracy : 0.903


## 応用レシピ

### (1) 回答の長さの制御

回答の長さを制御する指示を与えて文章を生成します。

In [ ]:
prompt = """
### 指示:
以下の質問に答えてください。

### 入力:
アマゾン川はどこにありますか？

### 回答:
"""

print(generate(prompt, True, 256))


### 指示:
以下の質問に答えてください。

### 入力:
アマゾン川はどこにありますか？

### 回答:
アマゾン川は南米のブラジルとペルーの国境にあるアマゾン川の源流から始まり、南アメリカのブラジルの南東部を流れ、大西洋に注いでいます。<EOD|LLM-jp>


In [ ]:
prompt = """
### 指示:
以下の質問に20字以内で答えてください。

### 入力:
アマゾン川はどこにありますか？

### 回答:
"""

print(generate(prompt, True, 256))


### 指示:
以下の質問に20字以内で答えてください。

### 入力:
アマゾン川はどこにありますか？

### 回答:
ブラジルの南東部<EOD|LLM-jp>


### (2) 回答の形式の制御

回答の形式を指示するプロンプトを与えます。

In [ ]:
# InstructionTuningに含まれていない文言は再現できません
prompt = """
### 指示:
以下の質問に箇条書きで３つ答えてください。

### 入力:
ブラジルの川は？

### 回答:
"""

print(generate(prompt, True, 256))


### 指示:
以下の質問に箇条書きで３つ答えてください。

### 入力:
ブラジルの川は？

### 回答:
アマゾン川<EOD|LLM-jp>


### (3) 学習したタスクと同じタスクを解く

学習データに入っているタスクと同じ指示を与えて文章生成させます。

In [ ]:
prompt = """
### 指示:
入力されたワードを説明してください。

### 入力:
プログラミング

### 回答:
"""

print(generate(prompt, True, 256))


### 指示:
入力されたワードを説明してください。

### 入力:
プログラミング

### 回答:
プログラミング（Programming）とは、コンピュータに何らかの処理をさせるための手順を記述することである。プログラミング言語を用いて記述する。<EOD|LLM-jp>


In [ ]:
prompt = """
### 指示:
入力されたワードを説明してください。

### 入力:
大規模言語モデル

### 回答:
"""

print(generate(prompt, True, 256))


### 指示:
入力されたワードを説明してください。

### 入力:
大規模言語モデル

### 回答:
大規模言語モデル（だいきぼげんごモデル、）は、自然言語処理の分野で、大規模なデータセットを用いて、統計的機械学習手法を用いて、言語モデルを構築する手法である。<EOD|LLM-jp>
